# 03 — Validacion de Salida

**Objetivo**: Garantizar que las respuestas del agente cumplan con schemas Pydantic, implementar retry loops y guardrails de contenido.

## Contenido
1. Output validation con Pydantic
2. Retry loop en LangGraph (max 3 intentos)
3. Guardrails de contenido
4. Metricas: valido al primer intento vs retry vs fallo

In [ ]:
import os
import json
from typing import TypedDict, Annotated, Literal
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from pydantic import BaseModel, Field, field_validator

load_dotenv()

llm = ChatOpenAI(model="gpt-5-mini", temperature=0)

print("=" * 60)
print("VALIDACION DE SALIDA")
print("=" * 60)

## 1. Schemas Pydantic para Output Validation

In [ ]:
# ============================================================
# SCHEMAS DE VALIDACION
# ============================================================

class ComicResponse(BaseModel):
    """Respuesta validada sobre un comic."""
    personaje: str = Field(description="Nombre del personaje principal")
    arco_narrativo: str = Field(description="Nombre del arco o saga")
    resumen: str = Field(description="Resumen de la respuesta (50-200 palabras)")
    fuentes_citadas: list[str] = Field(description="Lista de fuentes o arcos referenciados", min_length=1)
    confianza: float = Field(description="Confianza en la respuesta (0.0 a 1.0)", ge=0.0, le=1.0)

    @field_validator("resumen")
    @classmethod
    def resumen_no_vacio(cls, v: str) -> str:
        if len(v.split()) < 10:
            raise ValueError("El resumen debe tener al menos 10 palabras")
        return v


# Test: el modelo puede generar respuestas estructuradas
structured_llm = llm.with_structured_output(ComicResponse)

response = structured_llm.invoke(
    "Describe el arco de Knightfall de Batman en detalle"
)

print("Respuesta validada:")
print(f"  Personaje: {response.personaje}")
print(f"  Arco: {response.arco_narrativo}")
print(f"  Resumen: {response.resumen[:100]}...")
print(f"  Fuentes: {response.fuentes_citadas}")
print(f"  Confianza: {response.confianza:.1%}")
print(f"  Tipo: {type(response).__name__}")

## 2. Retry Loop en LangGraph

Si la respuesta no pasa validacion, el agente reintenta con feedback sobre el error.

```
┌──────────┐     ┌───────────┐     ┌──────────┐
│ Generar  │────▶│ Validar   │────▶│ Decidir  │
└──────────┘     └───────────┘     └────┬─────┘
     ▲                                   │
     │           ┌───────────┐    valido  │  invalido
     └───────────│  Corregir │◀───────────┘
                 └───────────┘     (max 3)
```

In [ ]:
# ============================================================
# RETRY LOOP CON LANGGRAPH
# ============================================================

class ValidationState(TypedDict):
    messages: Annotated[list, add_messages]
    query: str
    respuesta_raw: str
    respuesta_validada: dict | None
    intentos: int
    errores: list[str]
    status: str  # "pending", "valid", "failed"


def nodo_generar(state: ValidationState) -> dict:
    """Genera una respuesta del LLM."""
    query = state["query"]
    errores_previos = state.get("errores", [])

    system = "Eres un experto en comics. Responde en espa\u00f1ol con informacion precisa."
    if errores_previos:
        feedback = "\n".join(f"- {e}" for e in errores_previos[-2:])
        system += f"\n\nINTENTOS ANTERIORES FALLARON. Corrige estos errores:\n{feedback}"

    response = llm.invoke([
        SystemMessage(content=system),
        HumanMessage(content=query),
    ])
    return {
        "respuesta_raw": response.content,
        "intentos": state.get("intentos", 0) + 1,
    }


def nodo_validar(state: ValidationState) -> dict:
    """Valida la respuesta contra el schema Pydantic."""
    try:
        response = structured_llm.invoke(
            f"Estructura esta informacion sobre comics:\n{state['respuesta_raw']}"
        )
        return {
            "respuesta_validada": response.model_dump(),
            "status": "valid",
        }
    except Exception as e:
        return {
            "errores": state.get("errores", []) + [str(e)],
            "status": "invalid",
        }


def decidir_retry(state: ValidationState) -> str:
    """Decide si reintentar o terminar."""
    if state.get("status") == "valid":
        return "fin"
    if state.get("intentos", 0) >= 3:
        return "fallo"
    return "reintentar"


# Construir grafo
graph = StateGraph(ValidationState)
graph.add_node("generar", nodo_generar)
graph.add_node("validar", nodo_validar)

graph.add_edge(START, "generar")
graph.add_edge("generar", "validar")
graph.add_conditional_edges("validar", decidir_retry, {
    "fin": END,
    "fallo": END,
    "reintentar": "generar",
})

retry_app = graph.compile()
print("Grafo de retry compilado")

In [ ]:
# ============================================================
# TEST DEL RETRY LOOP
# ============================================================

result = retry_app.invoke({
    "messages": [],
    "query": "Explica el arco de Venom en Spider-Man",
    "respuesta_raw": "",
    "respuesta_validada": None,
    "intentos": 0,
    "errores": [],
    "status": "pending",
})

print("=" * 60)
print("RESULTADO DEL RETRY LOOP")
print("=" * 60)
print(f"Status: {result['status']}")
print(f"Intentos: {result['intentos']}")
print(f"Errores acumulados: {len(result.get('errores', []))}")
if result.get("respuesta_validada"):
    print(f"\nRespuesta validada:")
    for k, v in result["respuesta_validada"].items():
        print(f"  {k}: {v}")

## 3. Guardrails de Contenido

Ademas de validacion de formato, necesitamos validar el **contenido** de la respuesta.

In [ ]:
# ============================================================
# GUARDRAILS DE CONTENIDO
# ============================================================

class ContentGuardrail(BaseModel):
    """Evaluacion de calidad del contenido."""
    es_relevante: bool = Field(description="La respuesta es relevante a la pregunta")
    contiene_alucinacion: bool = Field(description="La respuesta contiene informacion inventada")
    es_segura: bool = Field(description="La respuesta no contiene contenido danino")
    score_calidad: int = Field(description="Score de calidad 1-5", ge=1, le=5)


guardrail_llm = llm.with_structured_output(ContentGuardrail)


def evaluar_contenido(pregunta: str, respuesta: str) -> ContentGuardrail:
    """Evalua la calidad del contenido de una respuesta."""
    prompt = f"""Evalua la siguiente respuesta:

Pregunta: {pregunta}
Respuesta: {respuesta}

Determina:
1. Si la respuesta es relevante a la pregunta
2. Si contiene alucinaciones (informacion inventada o falsa)
3. Si es segura (no contiene contenido danino)
4. Score de calidad general (1-5)
"""
    return guardrail_llm.invoke(prompt)


# Test con respuesta correcta
eval_ok = evaluar_contenido(
    "Quien es el Joker?",
    "El Joker es el archivillano de Batman, conocido por su apariencia de payaso, su risa maniaca, y su filosofia del caos."
)
print("Evaluacion de respuesta correcta:")
print(f"  Relevante: {eval_ok.es_relevante}")
print(f"  Alucinacion: {eval_ok.contiene_alucinacion}")
print(f"  Segura: {eval_ok.es_segura}")
print(f"  Score: {eval_ok.score_calidad}/5")

# Test con respuesta alucinada
eval_bad = evaluar_contenido(
    "Quien es el Joker?",
    "El Joker fue creado en 1975 por Stan Lee como villano de Spider-Man. Su nombre real es Jack Napier Lopez y nacio en Mexico."
)
print("\nEvaluacion de respuesta alucinada:")
print(f"  Relevante: {eval_bad.es_relevante}")
print(f"  Alucinacion: {eval_bad.contiene_alucinacion}")
print(f"  Segura: {eval_bad.es_segura}")
print(f"  Score: {eval_bad.score_calidad}/5")

## 4. Metricas de Validacion

In [ ]:
# ============================================================
# METRICAS DE VALIDACION (batch)
# ============================================================

queries_test = [
    "Que es la Batcueva?",
    "Como funciona el sentido aracnido?",
    "Explica el arco de Year One.",
    "Que paso en Civil War con Spider-Man?",
    "Quien es Catwoman?",
]

resultados = {"primer_intento": 0, "retry": 0, "fallo": 0}

print("=" * 60)
print("BATCH VALIDATION")
print("=" * 60)

for query in queries_test:
    result = retry_app.invoke({
        "messages": [],
        "query": query,
        "respuesta_raw": "",
        "respuesta_validada": None,
        "intentos": 0,
        "errores": [],
        "status": "pending",
    })

    if result["status"] == "valid":
        if result["intentos"] == 1:
            resultados["primer_intento"] += 1
            tag = "1er intento"
        else:
            resultados["retry"] += 1
            tag = f"retry ({result['intentos']} intentos)"
    else:
        resultados["fallo"] += 1
        tag = "FALLO"

    print(f"  [{tag:15s}] {query}")

print(f"\nResumen:")
print(f"  Valido al 1er intento: {resultados['primer_intento']}/{len(queries_test)}")
print(f"  Necesito retry:        {resultados['retry']}/{len(queries_test)}")
print(f"  Fallo permanente:      {resultados['fallo']}/{len(queries_test)}")

## Takeaways

1. **Pydantic** valida estructura Y contenido de las respuestas del LLM
2. **Retry loops** mejoran la tasa de exito pero agregan latencia y costo
3. **Guardrails** son la ultima linea de defensa contra alucinaciones
4. En produccion, la combinacion schema + guardrail + retry da >95% de respuestas validas
5. Siempre definir un `max_retries` para evitar loops infinitos y costos descontrolados